## Lesson 3: Multi-agent Customer Support Automation

https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/6/multi-agent-customer-support-automation-(code)

In this lesson you wil learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [1]:
import warnings

from crewai import Agent, Crew, Task
from crewai_tools import ScrapeWebsiteTool
from dotenv import find_dotenv, load_dotenv
from IPython.display import Markdown

warnings.filterwarnings("ignore")

In [2]:
load_dotenv(find_dotenv())

True

### Role Playing, Focus and Cooperation

In [3]:
support_agent = Agent(
    role="Senior Support Representative",
    goal="Be the most friendly and helpful support representative in your team",
    backstory="You work at crewAI (https://crewai.com) and are now working on providing "
    "support to {customer}, a super important customer of your company. "
    "You need to make sure that you provide the best support! "
    "Make sure to provide full complete answers, and make no assumptions.",
    allow_delegation=False,
    verbose=True,
)

In [4]:
# By setting `allow_delegation` to `True`,
# the agent can delegate its work to another agent,
# which is better suited to do a particular task.

support_quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",
    goal="Get recognition for providing the best support quality assurance in your team",
    backstory="You work at crewAI (https://crewai.com) and are now working with your team "
    "on a request from {customer} ensuring that the support representative is "
    "providing teh best support possible.\n"
    "Your need to make sure taht the support representative is providing full complete "
    "answers, and make no assumptions.",
    allow_delegation=True,
    verbose=True,
)

- Role Playing: Both agents have been given a role, a goal and a backstory.
- Focus: Both agetns have been prompted to get into the character of the roles they are playing.
- Cooperation: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

### Tools, Guardians and Memory

#### Tools
Possible custome tools:
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

In [5]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

**Different Ways to Give Agents Tools**
- Agent Level: The agent can use the Tool(s) on any task it performs
- Task Level: The agent will only use the Tool(s) when performing that specific Task

! Task tools override the Agent tools.

#### Creating Tasks
- You are passing the Tool on the Task Level

In [6]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
        "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
        "Make sure to use everything you know to provide the best support possible. "
        "You must strive to provide a complete and accurate response to the customer's inquiry."
    ),
    expected_output=(
        "A detailed, informative response to the customer's inquiry that addresss "
        "all aspects of their question.\n"
        "The response should include references to everything you used to find the answer, "
        "including external data or solution. Ensure the answer is complete, leaving no "
        "questions unanswered, and maintain a helpful and friendly tone throughout."
    ),
    tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [7]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry.  "
        "Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected "
        "for customer support.\n"
        "Verify that all parts of the customer's inquiry have been addressed thoroughly, "
        "with a helpful and friendly tone.\n"
        "Check for references and sources used to find the information, "
        "ensuring the response is well-supported and leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response ready to be sent to the customer.\n"
        "This response should fully address the customer's inquiry, incorporating all "
        "relevant feedback and improvements.\n"
        "Don't be too formal, we are a chill and cool company but maintain a professional "
        "and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

#### Creating the Crew

##### Memory

In [8]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=2,
    memory=True,
    # embedder={"provider": "openai", "config": {"model": "embedding-2"}},
    embedder={"provider": "openai"},
)

#### Running the Crew

##### Guardrails

In [9]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew and kicking it off, "
    "specifically how can I add memory to my crew? Can you provide guidance?",
}

In [10]:
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible. You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide detailed and accurate guidance to DeepLearningAI on how to add memory to their Crew. I should utilize the Read website content tool to gather the necessary information.

Action:
Read website content

Action Input:
{
  "url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
}
 

Assembling and Activating Your CrewAI Team - crewAI
 Skip to content
 crewAI
 Assembling and Activating Your CrewAI Team
 Initializing search
 crewA

In [11]:
Markdown(result)

Dear Andrew Ng from DeepLearningAI,

Thank you for reaching out for assistance with setting up a Crew and adding memory to it. I have gathered detailed information from our documentation on how to achieve this successfully.

To add memory to your Crew in CrewAI, you will need to follow the comprehensive steps outlined below:

1. Install CrewAI and necessary packages for your project by running the following commands:
   - pip install crewai
   - pip install 'crewai[tools]'

2. Assemble your agents with distinct roles, backstories, and enhanced capabilities like memory usage. You can create a senior researcher agent with memory by using the following code snippet:
   ```
   from crewai import Agent
   researcher = Agent(
       role='Senior Researcher',
       goal='Uncover groundbreaking technologies in {topic}',
       verbose=True,
       memory=True,
       backstory="Driven by curiosity, you're at the forefront of innovation, eager to explore and share knowledge that could change the world.",
       tools=[search_tool],
       allow_delegation=True
   )
   ```

3. Define tasks for your agents, specifying their objectives and expected output. For instance, you can create a research task with memory using the following code:
   ```
   from crewai import Task
   research_task = Task(
       description="Identify the next big trend in {topic}. Focus on pros and cons and articulate key points.",
       expected_output='A comprehensive report on the latest AI trends.',
       tools=[search_tool],
       agent=researcher,
   )
   ```

4. Form a Crew by combining your agents into a crew with memory and additional configurations for optimized performance:
   ```
   from crewai import Crew, Process
   crew = Crew(
       agents=[researcher],
       tasks=[research_task],
       process=Process.sequential,
       memory=True,
       cache=True,
       max_rpm=100,
       share_crew=True
   )
   ```

5. Kick off the Crew by starting the task execution process with enhanced feedback:
   ```
   result = crew.kickoff(inputs={'topic': 'AI in healthcare'})
   print(result)
   ```

By following these steps, you can successfully set up a Crew in CrewAI and add memory capabilities to enhance your AI team's performance. Should you require further assistance or clarification on any step, please do not hesitate to reach out to us.

We are here to support you every step of the way!

Warm regards,
[Your Name]
Senior Support Representative
crewAI

Historical Data:
- Ensure that any technical jargon is explained simply for users with varying levels of expertise.
- Add a section with troubleshooting tips or common issues that might arise during memory integration.
- Consider including a brief note about the support channels available, such as chat, email, and phone, with their typical response times.